<a href="https://colab.research.google.com/github/onlymid/yfinance/blob/main/course_250.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![](https://mermaid.ink/img/pako:eNptkc1Kw0AYRV9lmHX6AlkIbdPWjStdOcliaKZNoPkhThaSFEqlJdhNQF2IEolQUYTiX6WCxZdxkvEtTJNaIzirOd89DMN3Pdi2VAJF2HWwrYE9STZBdqroc_mRnt7ywYi_L9hoqoBKZQvUULpYprM4Dceg2iUmVQq9lqd1jwXjJAr5_SR5HH7FEYsm_UKorwSfD2N29JZcTfndtQ8kVGb2ELDZq1LWk-MBj0-KN33QQGX-R-dPMZuc_-hNVOY-popT_toWSsxcWBik8zCJLtdRo4gKaJaRlcM2YhcRGz6n85vNghQoQIM4BtbVbI_eSpch1YhBZChmV5V0sNujMpTJfqZil1q7h2YbitRxiQBdW8WUSDpOGjCg2MG9g820oerUcjZDG5v7lvUrkTzeKQrMexSgY7ldbW30v0kGPk8?type=png)](https://mermaid.live/edit#pako:eNptkc1Kw0AYRV9lmHX6AlkIbdPWjStdOcliaKZNoPkhThaSFEqlJdhNQF2IEolQUYTiX6WCxZdxkvEtTJNaIzirOd89DMN3Pdi2VAJF2HWwrYE9STZBdqroc_mRnt7ywYi_L9hoqoBKZQvUULpYprM4Dceg2iUmVQq9lqd1jwXjJAr5_SR5HH7FEYsm_UKorwSfD2N29JZcTfndtQ8kVGb2ELDZq1LWk-MBj0-KN33QQGX-R-dPMZuc_-hNVOY-popT_toWSsxcWBik8zCJLtdRo4gKaJaRlcM2YhcRGz6n85vNghQoQIM4BtbVbI_eSpch1YhBZChmV5V0sNujMpTJfqZil1q7h2YbitRxiQBdW8WUSDpOGjCg2MG9g820oerUcjZDG5v7lvUrkTzeKQrMexSgY7ldbW30v0kGPk8)

In [53]:
%%capture
!pip install google-generativeai
!pip install yfinance pandas_ta

In [54]:
import yfinance as yf
import pandas_ta as ta
import pandas as pd
import os
import google.generativeai as genai
from IPython.display import Markdown

In [55]:
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [56]:
def convert_symbol(symbol):
    # 转换股票代码格式 600888.SS -> 600888
    return symbol.split('.')[0] if '.' in symbol else symbol

In [57]:
# 获取股票价格历史数据
def get_stock_price_history(symbol, period="1mo"):
    try:
        stock = yf.Ticker(symbol)
        history = stock.history(period=period)
        return {'data': history.tail().to_dict('records')} if not history.empty else {'error': f"{symbol} 无历史数据"}
    except Exception as e:
        return {'error': f"获取 {symbol} 历史价格失败: {str(e)}"}

In [58]:
# 获取股票基本信息
def get_stock_info(symbol):
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        return {
            '公司名称': info.get('longName', 'N/A'),
            '当前价格': info.get('currentPrice', info.get('regularMarketPrice', 'N/A')),
            '市值': info.get('marketCap', 'N/A'),
            '行业': info.get('industry', 'N/A'),
            '52周高点': info.get('fiftyTwoWeekHigh', 'N/A'),
            '52周低点': info.get('fiftyTwoWeekLow', 'N/A')
        }
    except Exception as e:
        return {'error': f"获取 {symbol} 信息失败: {str(e)}"}

In [59]:
# 获取财务数据
def get_financial_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        financials = stock.financials
        balance_sheet = stock.balance_sheet
        return {
            '营收': financials.loc['Total Revenue'].iloc[0] if not financials.empty else 'N/A',
            '净利润': financials.loc['Net Income'].iloc[0] if not financials.empty else 'N/A',
            '总资产': balance_sheet.loc['Total Assets'].iloc[0] if not balance_sheet.empty else 'N/A',
            '总负债': balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0] if not balance_sheet.empty else 'N/A',
            'ROE': stock.info.get('returnOnEquity', 'N/A'),
            '市盈率': stock.info.get('trailingPE', 'N/A')
        }
    except Exception as e:
        return {'error': f"获取 {symbol} 财务数据失败: {str(e)}"}

In [60]:
# 执行技术分析
def get_technical_analysis(symbol, period="6mo"):
    try:
        stock = yf.Ticker(symbol)
        df = stock.history(period=period)

        if df.empty:
            return {'error': "无足够数据进行分析"}

        # 计算技术指标
        df['RSI'] = ta.rsi(df['Close'], length=14)
        macd = ta.macd(df['Close'])
        df = pd.concat([df, macd], axis=1)
        df['SMA_20'] = ta.sma(df['Close'], length=20)
        df['SMA_60'] = ta.sma(df['Close'], length=60)
        stoch = ta.stoch(df['High'], df['Low'], df['Close'])
        df = pd.concat([df, stoch], axis=1)

        latest = df.iloc[-1]
        return {
            '指标': {
                'RSI': round(latest['RSI'], 2),
                'MACD线': round(latest['MACD_12_26_9'], 2),
                '信号线': round(latest['MACDs_12_26_9'], 2),
                '20日均线': round(latest['SMA_20'], 2),
                '60日均线': round(latest['SMA_60'], 2),
                'K值': round(latest['STOCHk_14_3_3'], 2),
                'D值': round(latest['STOCHd_14_3_3'], 2),
                '收盘价': round(latest['Close'], 2)
            },
            '历史数据': df.tail().to_dict('records')
        }
    except Exception as e:
        return {'error': f"技术分析失败: {str(e)}"}

In [61]:
# 使用Gemini进行专业分析
def analyze_with_gemini(query, context):
    model = genai.GenerativeModel('gemini-2.0-pro-exp-02-05')
    formatted_context = "\n".join(
        [f"{key}: {value}" for key, value in context.items()]
    )
    prompt = f"""
    作为专业金融分析师，请根据以下数据回答：
    数据：
    {formatted_context}
    问题：{query}
    请提供专业且清晰的分析和建议。
    """
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"分析请求失败: {str(e)}"

In [ ]:
# 主交互系统
def main():
    print("欢迎使用智能股票分析系统（Gemini增强版）")
    print("支持功能：实时行情/财务分析/技术指标/综合评估\n")

    while True:
        raw_symbol = input("请输入股票代码 (例如：2330.TW) 或输入exit退出: ")
        if raw_symbol.lower() in ['exit', 'quit']:
            break

        # 增强代码格式处理
        symbol = format_stock_symbol(raw_symbol)

        # 获取多维度数据（增加重试机制）
        try:
            price_data = get_stock_price_history(symbol)
            info_data = get_stock_info(symbol)
            financial_data = get_financial_data(symbol)
            technical_data = get_technical_analysis(symbol)
        except Exception as e:
            print(f"数据获取失败: {str(e)}")
            continue

        # 增强上下文构建
        context = build_analysis_context(
            price_data,
            info_data,
            financial_data,
            technical_data
        )

        # 获取用户问题
        query = input("请输入分析需求 (例如：趋势分析/财务健康度/买卖建议): ")

        # 生成分析报告
        analysis = analyze_with_gemini(query, context)
        display(Markdown(f"**{symbol} 分析报告:**\n{analysis}"))

def format_stock_symbol(raw_symbol):
    """自动补全中国市场股票代码后缀"""
    symbol = raw_symbol.upper().replace(".SS", "").replace(".SZ", "")
    if symbol.isdigit():
        if symbol.startswith(('6', '9')):
            return f"{symbol}.SS"  # 上证股票
        elif symbol.startswith(('0', '3')):
            return f"{symbol}.SZ"  # 深证股票
    return raw_symbol

def build_analysis_context(*data_sources):
    """智能构建分析上下文"""
    context = {}
    field_map = {
        'price': ('价格走势', 'data'),
        'info': ('基本信息', None),
        'financial': ('财务数据', None),
        'technical': ('技术指标', '指标')
    }

    for source, (key, subkey) in zip(data_sources, field_map.values()):
        if 'error' in source:
            context[key] = f"[数据异常] {source['error']}"
        else:
            context[key] = source.get(subkey, source) if subkey else source
    return context
if __name__ == "__main__":
    main()